In [1]:
import gc
import IPython
import ipywidgets as widgets
import os
import glob
import shutil
import nibabel as nii
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
from ipywidgets import interactive
from IPython.display import display, Markdown, clear_output, Javascript, set_matplotlib_close

set_matplotlib_close(True)

#sns.set_theme()

%matplotlib inline
plt.rcParams["axes.grid"] = False

# Display some tests

img = nii.load('./arwibo_output_2/T1w/nG_ARWIBO_10002_T1w.nii.gz').get_fdata()

x_slice = img[img.shape[0] // 2]
y_slice = img[:, img.shape[1] // 2]
z_slice = img[:, :, img.shape[2] // 2]

slices = [x_slice, y_slice, z_slice]

fig, axes = plt.subplots(1, len(slices), figsize=(18, 6))
fig.suptitle('nG_ARWIBO_10002_T1w.nii.gz')
for i, slice in enumerate(slices):
    if i == 0:
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
    else:
        axes[i].imshow(slice.T, cmap="gray", origin="lower", extent=[0,100,0,1], aspect=175)

# Quality control

In [2]:
n = 1057

In [3]:
def f(name, image_type):
    def on_button_clicked(_):
        # "linking function with output"
        with out:
            # what happens when we press the button
            image_selected = name
            
            print(f"""n = {globals()['n']}""")
            print(image_selected)
            
            globals()['n'] += 1
            globals()['button'].close()
            
            image_selected_id = image_selected.split('_')[2]
            image_selected_type = 'T1w' if 'T1w' in image_selected.split('_')[3] else 'FLAIR'

            existed_images = glob.glob(f'./output2/{image_type}/nG_ARWIBO_{image_selected_id}_{image_selected_type}*.gz')
            print(f'./output2/{image_type}/{image_selected}')
            existed_images.remove(f'./output2/{image_type}/{image_selected}')
            print(existed_images)

            for image in existed_images:
                os.remove(image)

            shutil.move(f'./output2/{image_selected_type}/{image_selected}',
                        f'./output2/{image_selected_type}/ARWIBO_{image_selected_id}_{image_selected_type}.nii.gz')
            
    return on_button_clicked

In [4]:
# unique mark

try:
    img1 = glob.glob('./arwibo/*')[n]
    img1 = os.path.basename(img1)
    img1.split('+')
    img_id = img1.split('+')[2]
    img_info = img1.split('+')[6]
    image_type = ""
    if img_info in ['FLAIR', 'T2FL', 'T2FLV01', 'T2FLV02']:
        image_type = "FLAIR"
    elif img_info in ['MPR', 'T13D', 'T13DV01', 'T13DV02', 'T1TSE']:
        image_type = "T1w"
    
    if image_type == "":
        raise NotImplementedError
    if len(glob.glob(f'./output2/{image_type}/nG_ARWIBO_{img_id}_{image_type}*.gz')) == 0:
        raise NotImplementedError
        
    clear_output()
    for img_path in glob.glob(f'./output2/{image_type}/nG_ARWIBO_{img_id}_{image_type}*.gz'):
        if len(glob.glob(f'./output2/{image_type}/nG_ARWIBO_{img_id}_{image_type}*.gz')) == 1:
            print(n, f"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t{os.path.basename(img_path)}")
        else:
            print(n, f"\x1b[31m\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t{os.path.basename(img_path)}\x1b[0m")

        img = nii.load(img_path).get_fdata()
        
        if np.min(img.shape) > 80:
            if image_type == "FLAIR":
                print("\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\x1b[31mATTENTION\x1b[0m")
        else:
            if image_type == "T1w":
                print("\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\x1b[31mATTENTION\x1b[0m")

        x_slice = img[img.shape[0] // 2]
        y_slice = img[:, img.shape[1] // 2]
        z_slice = img[:, :, img.shape[2] // 2]

        slices = [x_slice, y_slice, z_slice]
        fig, axes = plt.subplots(1, len(slices), figsize=(18, 6), clear=True)
        fig.suptitle(os.path.basename(img_path))

        for i, slice in enumerate(slices):
            if i == 0:
                axes[i].imshow(slice.T, cmap="gray", origin="lower")
            else:
                axes[i].imshow(slice.T, cmap="gray", origin="lower", extent=[0,100,0,1], aspect=175)
        plt.show()

        button = widgets.Button(description=os.path.basename(img_path))
        out = widgets.Output()
        button.on_click(f(os.path.basename(img_path), image_type))

        display(widgets.VBox([button,out]))
except:
    n += 1
    print(n)

1058


# 4D MRI

In [5]:
def f2(img_path):
    def on_button_clicked(_):
        # "linking function with output"
        with out:
            globals()['button'].close()
            globals()['button_delete'].close()
            img_header = nii.load(img_path).header
            img_affine = nii.load(img_path).affine
            globals()['img'] = nii.Nifti1Image(globals()['img'], img_affine, img_header)
            globals()['img'].to_filename(os.path.join('./output2/FLAIR/', os.path.basename(img_path)[3:]))
            os.remove(img_path)          
    return on_button_clicked

In [6]:
def delete(img_path):
    def on_button_clicked(_):
        # "linking function with output"
        globals()['button'].close()
        globals()['button_delete'].close()
        with out:
            os.remove(img_path)          
    return on_button_clicked

In [7]:
# u mark 2
print(len(glob.glob('./output2/FLAIR/nG_*')))

img_path = glob.glob('./output2/FLAIR/nG_*')[0]
print(img_path)
img = nii.load(img_path).get_fdata()
print(img.shape)
shape4 = False
if len(img.shape) == 4:
    img = np.concatenate(list(img.transpose([3, 0, 1, 2]))[::-1], axis=-1)
    shape4 = True
    
x_slice = img[img.shape[0] // 2]
y_slice = img[:, img.shape[1] // 2]
z_slice = img[:, :, img.shape[2] // 2]

slices = [x_slice, y_slice, z_slice]
fig, axes = plt.subplots(1, len(slices), figsize=(18, 6), clear=True)
fig.suptitle(os.path.basename(img_path))

for i, slice in enumerate(slices):
    if i == 0:
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
    else:
        axes[i].imshow(slice.T, cmap="gray", origin="lower", extent=[0,100,0,1], aspect=175)
plt.show()

if shape4:
    button = widgets.Button(description=f"SAVE", button_style='warning')
else:
    button = widgets.Button(description=f"SAVE", button_style='info')
out = widgets.Output()
button.on_click(f2(img_path))

button_delete = widgets.Button(description="DELETE", button_style='danger')
out_delete = widgets.Output()
button_delete.on_click(delete(img_path))

display(widgets.VBox([button,out]))
display(widgets.VBox([button_delete,out_delete]))

0


IndexError: list index out of range

# Filter metadata

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('./arwibo.csv')
df.head()

,ID_SOGGETTO,STUDY_NAME,AGE_BASELINE,GENDER,AGE,TIME_POINT,DIAGNOSIS,MMSE_tot,APOE,PTAU,TTAU,AB42,IMAGE_INFO,IMAGE_TYPE
0,1001,ARWIBO,70,F,70,T00,CN,25,e3e3,NaN,NaN,NaN,T2FL,FLAIR
1,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T1G2D,T12D
2,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T2FL,FLAIR
3,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T13D,T13D
4,1008,ARWIBO,52,F,52,T00,CN,27,e3e3,NaN,NaN,NaN,T2FL,FLAIR


In [10]:
df.IMAGE_TYPE.unique()

array(['FLAIR', 'T12D', 'T13D'], dtype=object)

In [11]:
df.GENDER.unique()

array(['F', 'M'], dtype=object)

In [23]:
print(len(df))

3481


In [12]:
df['sex'] = ''
df.sex[df.GENDER == 'F'] = 'female'
df.sex[df.GENDER == 'M'] = 'male'
df.head()

/home/huydnguyen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/huydnguyen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID_SOGGETTO,STUDY_NAME,AGE_BASELINE,GENDER,AGE,TIME_POINT,DIAGNOSIS,MMSE_tot,APOE,PTAU,TTAU,AB42,IMAGE_INFO,IMAGE_TYPE,sex
0,1001,ARWIBO,70,F,70,T00,CN,25,e3e3,NaN,NaN,NaN,T2FL,FLAIR,female
1,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T1G2D,T12D,male
2,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T2FL,FLAIR,male
3,1002,ARWIBO,61,M,61,T00,CN,30,e3e3,NaN,NaN,NaN,T13D,T13D,male
4,1008,ARWIBO,52,F,52,T00,CN,27,e3e3,NaN,NaN,NaN,T2FL,FLAIR,female


In [27]:
T1w_tosave = pd.DataFrame(columns=['filename', 'age', 'sex', 'diagnosis'])
for i in range(len(df)):
    if df.iloc[i].IMAGE_TYPE == 'T13D':
        if os.path.exists(f'./arwibo_output/T1w/ARWIBO_{df.iloc[i].ID_SOGGETTO}_T1w.nii.gz'):
            T1w_tosave = T1w_tosave.append(
                {
                    'filename': f'ARWIBO_{df.iloc[i].ID_SOGGETTO}_T1w.nii.gz',
                    'age': df.iloc[i].AGE,
                    'sex': df.iloc[i].sex,
                    'diagnosis': df.iloc[i].DIAGNOSIS
                },
                ignore_index=True
            )
T1w_tosave = T1w_tosave.drop_duplicates()
T1w_tosave.to_csv('t1w_info.csv', index=False)
T1w_tosave.head()

,filename,age,sex,diagnosis
0,ARWIBO_1002_T1w.nii.gz,61,male,CN
1,ARWIBO_1014_T1w.nii.gz,61,female,CN
2,ARWIBO_1016_T1w.nii.gz,42,female,CN
3,ARWIBO_1019_T1w.nii.gz,44,male,CN
4,ARWIBO_1043_T1w.nii.gz,45,male,CN


In [28]:
list_t1 = glob.glob('./arwibo_output/T1w/*')
for e in list_t1:
    if T1w_tosave['filename'].str.contains(os.path.basename(e)).sum() != 1:
        print(os.path.basename(e), T1w_tosave['filename'].str.contains(os.path.basename(e)).sum())

In [29]:
Flair_tosave = pd.DataFrame(columns=['filename', 'age', 'sex', 'diagnosis'])
for i in range(len(df)):
    if df.iloc[i].IMAGE_TYPE == 'FLAIR':
        if os.path.exists(f'./arwibo_output/FLAIR/ARWIBO_{df.iloc[i].ID_SOGGETTO}_FLAIR.nii.gz'):
            Flair_tosave = Flair_tosave.append(
                {
                    'filename': f'ARWIBO_{df.iloc[i].ID_SOGGETTO}_FLAIR.nii.gz',
                    'age': df.iloc[i].AGE,
                    'sex': df.iloc[i].sex,
                    'diagnosis': df.iloc[i].DIAGNOSIS
                },
                ignore_index=True
            )
Flair_tosave = Flair_tosave.drop_duplicates()
Flair_tosave.to_csv('flair_info.csv', index=False)
Flair_tosave.head()

,filename,age,sex,diagnosis
0,ARWIBO_1001_FLAIR.nii.gz,70,female,CN
1,ARWIBO_1002_FLAIR.nii.gz,61,male,CN
2,ARWIBO_1008_FLAIR.nii.gz,52,female,CN
3,ARWIBO_1014_FLAIR.nii.gz,61,female,CN
4,ARWIBO_1016_FLAIR.nii.gz,42,female,CN


In [30]:
list_flair = glob.glob('./arwibo_output/FLAIR/*')
for e in list_flair:
    if Flair_tosave['filename'].str.contains(os.path.basename(e)).sum() != 1:
        print(os.path.basename(e), Flair_tosave['filename'].str.contains(os.path.basename(e)).sum())